In [62]:
import numpy as np
import pandas as pd
import re
import time
import pickle
from glob import glob

from selenium import webdriver

from hackaroutine import Ingredient, Product, Routine

In [2]:
# CHROMEDRIVER_PATH = '/Users/Jelly/Documents/googledrive/chromedriver/macos/chromedriver'
CHROMEDRIVER_PATH = 'C:/chromedriver_win32/chromedriver.exe'

In [3]:
driver = webdriver.Chrome(CHROMEDRIVER_PATH)

In [4]:
file_list = glob('./data/*.csv')
file_list

['./data\\brands.csv',
 './data\\recommendations_2020-05-05.csv',
 './data\\responses_2020-05-05.csv',
 './data\\responses_modified_2020-05-05.csv']

In [55]:
responses = pd.read_csv(file_list[3])
responses.head(2)

,Timestamp,What's your skin type?,Do you have any known allergies? Or any ingredients you avoid for any reason?,What are your top skin concerns?,What is your current day routine? (Please list your products in order from first to last),What is your current night routine (Please list your products in order from first to last),What type of summer weather will you have?,Any known skin sensitivities?,"Any experience with retinoids? (If so, please tell us your use history)","Any experience with acids? (If so, please tell us your use history, what type, and any known concentrations)","Are you prone to breakouts? If so, how severe are the breakouts and what is your acne treatment history?",Anything else you want the chemists to know about your skin and your skincare routine?,"Can we share your routine learnings (anonymously) on our social media (post, podcast, or insta live shoutout)?"
0,4/30/2020 12:43:32,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","trader joe's all in one cleanser, bioderma seb...","trader joe's all in one cleanser, Bioderma Seb...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...","Biore Charcoal Acne Clearing Cleanser, Kiel's ...",NaN,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes


In [56]:
new_columns = [
    'timestamp',
    'skin_type',
    'cannot_contain',
    'skin_concern',
    'am_routine',
    'pm_routine',
    'climate',
    'skin_sensitivities',
    'used_retinoids',
    'used_acids',
    'prone_to_breakouts',
    'miscellaneous',
    'permission'
]

responses.columns = new_columns
responses.head(2)

,timestamp,skin_type,cannot_contain,skin_concern,am_routine,pm_routine,climate,skin_sensitivities,used_retinoids,used_acids,prone_to_breakouts,miscellaneous,permission
0,4/30/2020 12:43:32,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","trader joe's all in one cleanser, bioderma seb...","trader joe's all in one cleanser, Bioderma Seb...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...","Biore Charcoal Acne Clearing Cleanser, Kiel's ...",NaN,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes


In [58]:
routine_columns = ['am_routine', 'pm_routine']

for col in routine_columns:
    responses[col] = responses[col].str.lower().str.split(',')
    print(responses[col][:2])

0    [trader joe's all in one cleanser,  bioderma s...
1    [biore charcoal acne clearing cleanser,  kiel'...
Name: am_routine, dtype: object
0    [trader joe's all in one cleanser,  bioderma s...
1                                                  NaN
Name: pm_routine, dtype: object


In [59]:
responses['am_routine'][0]

["trader joe's all in one cleanser",
 ' bioderma sebium toner',
 ' the ordinary ascorbic acid 8%',
 ' bioderma sebium mat control cream',
 ' purlisse bb cream']

In [9]:
# prior experience

popular_brands = [
    'La Roche Posay',
    'Cerave', 
    'Cetaphil', 
    'Timeless Skincare', 
    'Skinceuticals', 
    'Stratia Skincare', 
    'Paula’s Choice', 
    'CosRX', 
    'The Ordinary', 
    'Drunk Elephant', 
    'Klairs', 
    'Inkey List', 
    'Farmacy ', 
    'First Aid Beauty'
]

popular_ingredients = [
    'Vitamin C', 
    'Glycolic Acid', 
    'Lactic Acid', 
    'Mandelic Acid', 
    'Niacinamide', 
    'Retinol', 
    'Hydroxypinocolone Retinoate', 
    'Bakuchiol', 
    'Retinyl Palmitate', 
    'Ceramides', 
    'Salicylic Acid', 
    'Willow bark Extract', 
    'Adapalene', 
    'Sodium Lauryl Sulfate', 
    'Cocoamidopropyl Betaine', 
]

In [10]:
routine = Routine(name=0, routine=responses['am_routine'][0])

In [11]:
routine.link_sync()
print(routine.routine)

No results for bioderma sebium toner on CosDNA.
Enter new search:


(to skip search, enter 'SKIP' w/o quotes) SKIP


SKIP: bioderma sebium toner
No results for bioderma sebium mat control cream on CosDNA.
Enter new search:


(to skip search, enter 'SKIP' w/o quotes) bioderma sebium mat control


["trader joe's all in one cleanser", 'SKIP: bioderma sebium toner', 'the ordinary ascorbic acid 8%', 'bioderma sébium mat control', 'purlisse bb cream']


In [12]:
routine.top

[('water', 3),
 ('phenoxyethanol', 3),
 ('glycerin', 2),
 ('unavailable', 2),
 ('cyclopentasiloxane', 2),
 ('butylene glycol', 2),
 ('sodium chloride', 2),
 ('ethylhexyl methoxycinnamate', 2),
 ('titanium dioxide', 2),
 ('zinc oxide', 2)]

In [13]:
routine.products[2].ingredients

['propanediol', 'ascorbic acid (8%)', 'alpha-arbutin (2%)']

In [14]:
for ingredient in popular_ingredients:
    print(ingredient)
    print(routine.has(ingredient))
    print()

Vitamin C
['the ordinary ascorbic acid 8%']

Glycolic Acid
Routine does not have Glycolic Acid.
[]

Lactic Acid
Routine does not have Lactic Acid.
[]

Mandelic Acid
Routine does not have Mandelic Acid.
[]

Niacinamide
Routine does not have Niacinamide.
[]

Retinol
Routine does not have Retinol.
[]

Hydroxypinocolone Retinoate
Routine does not have Hydroxypinocolone Retinoate.
[]

Bakuchiol
Routine does not have Bakuchiol.
[]

Retinyl Palmitate
["trader joe's all in one cleanser"]

Ceramides
Routine does not have Ceramides.
[]

Salicylic Acid
['bioderma sébium mat control']

Willow bark Extract
Routine does not have Willow bark Extract.
[]

Adapalene
Routine does not have Adapalene.
[]

Sodium Lauryl Sulfate
Routine does not have Sodium Lauryl Sulfate.
[]

Cocoamidopropyl Betaine
["trader joe's all in one cleanser"]



In [16]:
routine._routine_dict

{'90172810251': 'water',
 '8f2df68856': 'peg-80 sorbitan laurate',
 '9a5810119': 'cocamidopropyl betaine',
 'f3bac6962': 'sodium trideceth sulfate',
 '04e7dc953': 'sodium lauroamphoacetate',
 '4548ae2101': 'peg 150 distearate',
 '48257f1077': 'ubiquinone',
 'ee1a1a373': 'tocopheryl acetate',
 'f246bf490': 'thioctic acid',
 '2ac38b321': 'retinyl palmitate',
 'fcf8b5303': 'phytonadione',
 'a83b7c483': 'cholecalciferol',
 'fe279c1318': 'ascorbyl palmitate',
 'b8a3bf116': 'citric acid',
 'fc86a9293': 'panthenol',
 'e812b9352': 'sorbic acid',
 'f09549300': 'phenoxyethanol',
 '2f5a452621': 'sodium lauroyl glutamate',
 '205eb9166': 'glycerin',
 '80ae0e12197': 'sodium lauroyl oat amino acids',
 'c0304615299': 'camellia sinensis leaf extract',
 'unavailable': 'unavailable',
 '72a94a19514': 'propanediol',
 '7927f4219': 'ascorbic acid (8%)',
 'd828e611478': 'alpha-arbutin (2%)',
 'bd4d8e1208': 'cyclopentasiloxane',
 'c7c7fe1947': 'methyl methacrylate crosspolymer',
 'b05d771602': 'dipropylene gly

In [66]:
with open('./data/master_dict.pickle', 'rb') as handle:
    master_dict = pickle.load(handle)

In [60]:
def ingredient_dict_builder(routine_dict, master_dict):
    base_url = 'https://cosdna.com/eng/'
    filtered_dict = {k:v for k,v in routine_dict.items() if k not in master_dict.keys()}
    for k, v in filtered_dict.items():
        driver.get(base_url + k + '.html')
        print(f'{k}: {v}')
        ingredient = input('preferred name: ')
        if ingredient == 'BREAK':
            break
        elif ingredient == '':
            master_dict[k] = v
        else:
            master_dict[k] = ingredient
    return master_dict

In [61]:
am_routines = []
for rout in responses['am_routine']:
    routine = Routine(name=i, routine=responses['am_routine'][i])
    routine.link_sync()
#     master_dict = ingredient_dict_builder(routine._routine_dict, master_dict)
    am_routines.append(routine)

No results for  bioderma sebium toner on CosDNA.
Enter new search:


(to skip search, enter 'SKIP' w/o quotes) bioderma sebium h2o


No results for  bioderma sebium mat control cream on CosDNA.
Enter new search:


(to skip search, enter 'SKIP' w/o quotes) bioderma sebium mat control


No results for  kiel's powerful-strength line-reducing concentrate 12.5% vitamin c serum on CosDNA.
Enter new search:


...

preferred name:  


f9054c1161: boron nitride


preferred name:  


339cf61617: disodium phosphate


preferred name:  


In [68]:
for i in range(5, 10):
    routine = Routine(name=i, routine=responses['am_routine'][i])
    routine.link_sync()
    master_dict= ingredient_dict_builder(routine._routine_dict, master_dict)

No results for seventh sense morning dew moisturizer on CosDNA.
Enter new search:


(to skip search, enter 'SKIP' w/o quotes) seventh sense morning dew


No results for  seventh sense inner light eye cream on CosDNA.
Enter new search:


(to skip search, enter 'SKIP' w/o quotes) SKIP


SKIP:  seventh sense inner light eye cream
No results for  jimbo & jules' bee buzzin' salve on CosDNA.
Enter new search:


...

preferred name:  


8abc216434: polysorbate 60


preferred name:  


0beac113927: physalis angulata extract


preferred name:  


In [69]:
master_dict

{'90172810251': 'water',
 '8f2df68856': 'peg-80 sorbitan laurate',
 '9a5810119': 'cocamidopropyl betaine',
 'f3bac6962': 'sodium trideceth sulfate',
 '04e7dc953': 'sodium lauroamphoacetate',
 '4548ae2101': 'peg 150 distearate',
 '48257f1077': 'coq10',
 'ee1a1a373': 'vitamin e acetate',
 'f246bf490': 'alpha-lipoic acid',
 '2ac38b321': 'vitamin a palmitate',
 'fcf8b5303': 'vitamin k1',
 'a83b7c483': 'vitamin d3',
 'fe279c1318': 'vitamin c palmitate',
 'b8a3bf116': 'citric acid',
 'fc86a9293': 'panthenol',
 'e812b9352': 'sorbic acid',
 'f09549300': 'phenoxyethanol',
 '2f5a452621': 'sodium lauroyl glutamate',
 '205eb9166': 'glycerin',
 '80ae0e12197': 'sodium lauroyl oat amino acids',
 'c0304615299': 'tea leaf extract',
 'unavailable': 'unavailable',
 '72a94a19514': 'propanediol',
 '7927f4219': 'vitamin c',
 'd828e611478': 'alpha-arbutin',
 'bd4d8e1208': 'cyclopentasiloxane',
 'c7c7fe1947': 'methyl methacrylate crosspolymer',
 'b05d771602': 'dipropylene glycol',
 'e530df6438': 'polymethylsi

In [70]:
# saving pickle
with open('./data/master_dict.pickle', 'wb') as handle:
    pickle.dump(master_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)